In [24]:
import os
import pandas as pd

root_path = "log_files"

all_data = []
start_kw = "Starting a new Game, game #"
day_kw = "Day: "
stats_kw = "Statistics: "
campaign_kw = "[startDay ="
qs_hw = "Quality Score"
profit_kw = "Cumulative Profit: "

folders = [i for i in os.listdir(root_path) if i[0] != "."]
print(folders)
for folder in folders:
    file_names = os.listdir(os.path.join(root_path, folder))
    for file_ in file_names:
        full_path = os.path.join(root_path, folder, file_)
        txt_num = file_.split(".")[0]

        with open(full_path, "r") as file:
            data_ = file.readlines()

            curr_game = 0
            curr_day = 0

            for line_ in data_:
                if start_kw in line_:
                    curr_game = int(
                        line_[line_.index(start_kw)+len(start_kw):])
                if day_kw in line_:
                    curr_day = int(
                        line_[line_.index(day_kw)+len(day_kw):-2])
                if stats_kw in line_:
                    if "null" in line_:
                        curr_stats = 0
                        price = 0
                    else:
                        curr_stats = line_[line_.index(stats_kw) + len(stats_kw):-1]
                        no_impressions = curr_stats[curr_stats.index("(") + 1:curr_stats.index(",")]
                        no_payment = curr_stats[curr_stats.index(",") + 2:curr_stats.index(")")]
                        if float(no_impressions) == 0:
                            price = 0
                        else:
                            price =  float(no_payment) / float(no_impressions)
                        
                if campaign_kw in line_:
                    curr_campaign = line_[line_.index(campaign_kw):]
                    curr_segment = curr_campaign[curr_campaign.index("Segment = ") + len("Segment = "): 
                                                curr_campaign.index("Reach = ")-2]
                if qs_hw in line_:
                    curr_qs = line_[line_.index(qs_hw) + len(qs_hw):-1]
                if profit_kw in line_:
                    curr_profit = line_[line_.index(profit_kw) + len(profit_kw):-1]
                    all_data.append([txt_num, curr_game, curr_day, curr_stats, price, curr_segment, curr_qs, curr_profit])

['2020-04-28', '2020-04-26', '2020-04-27', '2020-04-29', '2020-04-25', '2020-05-01', '2020-04-30']


In [25]:
df = pd.DataFrame(all_data, columns = ["txt_num", "game", "day", "stats", "price_index", "segment", "quality_score", "cumulative_profit"]) 

In [26]:
df[df["stats"] != 0]

,txt_num,game,day,stats,price_index,segment,quality_score,cumulative_profit
1,2200,1,2,"{3=(1837, 0.0)},",0.000000,MALE_OLD_LOW_INCOME,= 1.0,787.7478972431743
2,2200,1,3,"{3=(1845, 92.2499999999969)},",0.050000,MALE_OLD_LOW_INCOME,= 1.0,3585.372304865352
3,2200,1,4,"{3=(1813, 167.87162592795502)},",0.092593,MALE_OLD_LOW_INCOME,= 1.1101766763089793,4339.390160089134
12,2200,2,2,"{3=(594, 594.0)},",1.000000,FEMALE_YOUNG_LOW_INCOME,= 0.9999805623844686,-0.02309188725121203
23,2200,3,2,"{3=(2367, 0.0)},",0.000000,FEMALE_OLD,= 1.0,842.2061258902104
...,...,...,...,...,...,...,...,...
55507,1300,47,2,"{7=(1174, 1174.0)},",1.000000,FEMALE_OLD_LOW_INCOME,= 1.0,-668.7417114255531
55508,1300,47,3,"{7=(247, 19.760489343089386)},",0.080002,FEMALE_OLD_LOW_INCOME,= 0.663669661145653,-407.8187765216638
55518,1300,48,2,"{7=(2603, 784.0)},",0.301191,OLD_LOW_INCOME,= 1.1471330776412647,844.186823345422
55529,1300,49,2,"{7=(397, 397.0)},",1.000000,MALE_HIGH_INCOME,= 0.9999805623844686,-0.015433466731906265


In [42]:
df[df["stats"] != 0].groupby(['segment']).price_index.mean().to_dict()

{'FEMALE_HIGH_INCOME': 0.2725973382129604,
 'FEMALE_LOW_INCOME': 0.26272734690073507,
 'FEMALE_OLD': 0.259168408781004,
 'FEMALE_OLD_HIGH_INCOME': 0.299643364613246,
 'FEMALE_OLD_LOW_INCOME': 0.33007089761341973,
 'FEMALE_YOUNG': 0.31029370908681825,
 'FEMALE_YOUNG_HIGH_INCOME': 0.3452796419787934,
 'FEMALE_YOUNG_LOW_INCOME': 0.31504007991352573,
 'MALE_HIGH_INCOME': 0.2943665425899583,
 'MALE_LOW_INCOME': 0.3022844394912688,
 'MALE_OLD': 0.2774527289563977,
 'MALE_OLD_HIGH_INCOME': 0.32291884677035093,
 'MALE_OLD_LOW_INCOME': 0.3071734228588909,
 'MALE_YOUNG': 0.3024178329232215,
 'MALE_YOUNG_HIGH_INCOME': 0.33670612474460315,
 'MALE_YOUNG_LOW_INCOME': 0.32195648414764966,
 'OLD_HIGH_INCOME': 0.3086182161049369,
 'OLD_LOW_INCOME': 0.28692185865409875,
 'YOUNG_HIGH_INCOME': 0.28851944966548354,
 'YOUNG_LOW_INCOME': 0.28686355846427}

In [40]:
df[(df["stats"] != 0) & (df["price_index"] != 0)].groupby(['segment']).price_index.mean()

segment
FEMALE_HIGH_INCOME          0.390907
FEMALE_LOW_INCOME           0.376149
FEMALE_OLD                  0.369696
FEMALE_OLD_HIGH_INCOME      0.527892
FEMALE_OLD_LOW_INCOME       0.547593
FEMALE_YOUNG                0.430912
FEMALE_YOUNG_HIGH_INCOME    0.581860
FEMALE_YOUNG_LOW_INCOME     0.566635
MALE_HIGH_INCOME            0.403020
MALE_LOW_INCOME             0.430054
MALE_OLD                    0.390489
MALE_OLD_HIGH_INCOME        0.546896
MALE_OLD_LOW_INCOME         0.552478
MALE_YOUNG                  0.418220
MALE_YOUNG_HIGH_INCOME      0.569316
MALE_YOUNG_LOW_INCOME       0.559465
OLD_HIGH_INCOME             0.423613
OLD_LOW_INCOME              0.395428
YOUNG_HIGH_INCOME           0.409616
YOUNG_LOW_INCOME            0.387308
Name: price_index, dtype: float64